In [1]:
!pip install langchain langchain_community langchain_chroma langchain-huggingface langchain-mistralai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 90.8 MB/s eta 0:00:00
  

In [2]:
!pip install --upgrade --quiet  sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.7 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader
import os

file_path = os.path.join("artifacts","data_transformation","rag_input.csv")

loader = CSVLoader(file_path=file_path)
data = loader.load()

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(data)

In [5]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
persist_directory = os.path.join("artifacts","chroma_db")
vectorstore = Chroma.from_documents(
    splits,
    embedding=embeddings_model,
    persist_directory=persist_directory
)
vectorstore.persist()

In [ ]:

vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings_model)

In [8]:
retriever = vectordb.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 10},

)
query = "The Millionaire's AI Playbook: Learn How to Predict Stock Prices! (For Kids) lessons "
results = retriever.get_relevant_documents(query)

<ipython-input-8-7396009e26ca>:7: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)


In [10]:
results

[Document(metadata={'row': 2, 'source': '/content/rag_input.csv'}, page_content="input: The Millionaire's AI Playbook: Learn How to Predict Stock Prices! (For Kids) - lessons - 10"),
 Document(metadata={'row': 3, 'source': '/content/rag_input.csv'}, page_content="input: The Millionaire's AI Playbook: Learn How to Predict Stock Prices! (For Kids) - duration - 1 hour per session"),
 Document(metadata={'row': 4, 'source': '/content/rag_input.csv'}, page_content="input: The Millionaire's AI Playbook: Learn How to Predict Stock Prices! (For Kids) - price - 220"),
 Document(metadata={'row': 1, 'source': '/content/rag_input.csv'}, page_content="input: The Millionaire's AI Playbook: Learn How to Predict Stock Prices! (For Kids) - session price - 22"),
 Document(metadata={'row': 0, 'source': '/content/rag_input.csv'}, page_content="input: The Millionaire's AI Playbook: Learn How to Predict Stock Prices! (For Kids) - description - Learn to predict stock prices using the incredible potential of A